In [36]:
# Import libraries 
import os
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
import sklearn
from MCForecastTools  import MCSimulation

%matplotlib inline

In [37]:
# Load .env enviroment 
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")
#print(alpaca_api_key)
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)
print(api._api_version)

v2


In [38]:
# Set the ticker
ticker = "SQ"

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes 
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()



In [39]:
# Get  historical data 
ticker_data = api.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

ticker_data.head()

SQ                                     
                            open     high      low    close    volume
time                                                                 
2019-01-02 00:00:00-05:00  54.10  57.8300  53.5600  57.1300  12680319
2019-01-03 00:00:00-05:00  55.58  56.7300  52.2600  52.4711  18397355
2019-01-04 00:00:00-05:00  54.40  59.6600  53.8611  58.0215  29065071
2019-01-07 00:00:00-05:00  59.00  61.2900  58.0501  60.6200  16722213
2019-01-08 00:00:00-05:00  62.60  63.8002  61.1500  63.0500  20136281

### Run the Monte Carlo Simulation

In [40]:
# Set number of simulations
MC_sq = MCSimulation(
    portfolio_data = ticker_data,
    weights = [.60,.40],
    num_simulation = 500,
    num_trading_days = 252*2
)
# Configure a Monte Carlo simulation to forecast two year daily returns



In [41]:
# Run Monte Carlo simulations to 
# forecast one year daily returns
MC_sq.calc_cumulative_return(self)

Running Monte Carlo simulation number 0.


ValueError: Dot product shape mismatch, (505, 1) vs (2,)

### Plot the Simulated Outcomes for the Next Year (252 Traiding Days)

In [ ]:
# Plot simulation outcomes
line_plot = MC_sq.plot_simulation()

### Plot the Simulated Daily Returns Trajectory for `AAPL` over the Next Year (252 Trading Days)

In [ ]:
# Compute summary statistics
simulated_returns_data = {
    "mean": list(MC_sq.simulated_return.mean(axis=1)),
    "median": list(MC_sq.simulated_return.median(axis=1)),
    "min": list(MC_sq.simulated_return.min(axis=1)),
    "max": list(MC_sq.simulated_return.max(axis=1))
}

# Create a DataFrame with the summary statistics
df_simulated_returns = pd.DataFrame(simulated_returns_data)

# Display sample data
df_simulated_returns.head()

In [ ]:
# plot the trajectory of daily returns
df_simulated_returns.plot(title="Simulated Daily Returns Behavior of SQ Stock Over the Next Year")

In [ ]:
# Set initial investment
initial_investment = 10000

# Multiply an initial investment by the daily returns of simulative stock prices to return the progression of daily returns in terms of money
cumulative_pnl = initial_investment * df_simulated_returns

# Display sample data
cumulative_pnl.head()

In [ ]:
# plot of the simulated profits/losses
cumulative_pnl.plot(title="Simulated Outcomes Behavior of SQ Stock Over the Next Year")

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl = MC_sq.summarize_cumulative_return()

# Print summary statistics
print(tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments 
ci_lower = round(tbl[8]*10000,2)
ci_upper = round(tbl[9]*10000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $10,000 in the portfolio"
      f" over the next year will end within in the range of"
      f" ${ci_lower} and ${ci_upper}.")